In [1]:
%matplotlib inline
from os.path import join as pjoin
import matplotlib.pyplot as plt
from matplotlib import patheffects
import pandas as pd
import numpy as np
import seaborn as sns

from sktime.classification.kernel_based import RocketClassifier, Arsenal
from sklearn.preprocessing import StandardScaler

from stndata import ONEMINUTESTNNAMES

np.random.seed(1000)


Load the stations, and the manual storm classification data. That data is stored in a separate csv file, indexed by date and station number. We then split the storm data into a training set and a test set. Bear in mind this is still the subset of high quality stations. There's another 500-odd stations that we have processed to extract daily maximum wind gust data.

In [2]:
BASEDIR = r"X:\georisk\HaRIA_B_Wind\data\derived\obs\1-minute\events"

stndf = pd.read_csv(pjoin(BASEDIR, 'hqstations.csv'), index_col="stnNum")

eventFile = pjoin(BASEDIR, "CA_20230518_Hobart.csv")
#eventFile = pjoin(BASEDIR, "NA_all.csv")
stormdf = pd.read_csv(eventFile, usecols=[1, 2, 3], parse_dates=['date'],
                      dtype={'stnNum': int,
                             'stormType': 'category'})

#stormdf = pd.read_csv(eventFile, usecols=[2, 3, 4], parse_dates=['date'],
#                dtype={'stnNum': float,
#                       'stormType': 'category'},
#                converters={'stnNum': lambda s: int(float(s.strip() or 0))})

stormdf.set_index(['stnNum', 'date'], inplace=True)
nevents = len(stormdf)

# Take a random selection of 200 storms to test against:
test_storms = stormdf.sample(200)
train_storms = stormdf.drop(test_storms.index)
ntrain = len(train_storms)

In [8]:
import sklearn; sklearn.__version__

'1.1.3'

In [17]:
def loadData(stnNum: int) -> pd.DataFrame:
    """
    Load event data for a given station. Missing values are interpolated
    linearly - if values are missing at the start or end they are backfilled
    from the nearest valid value.

    This data has been extracted by `extractStationData.py`, and is stored in
    pickle files, so there should be no issues around type conversions.

    :param stnNum: BoM station number
    :type stnNum: int
    :return: DataFrame holding the data of all gust events for a station
    :rtype: `pd.DataFrame`
    """
    fname = pjoin(BASEDIR, "events", f"{stnNum:06d}.pkl")
    df = pd.read_pickle(fname)
    df['date'] = pd.to_datetime(df['date'])
    df['windgust'] = df['windgust'].interpolate(method='linear').fillna(method='bfill')
    df['tempanom'] = df['tempanom'].interpolate(method='linear').fillna(method='bfill')
    df['stnpanom'] = df['stnpanom'].interpolate(method='linear').fillna(method='bfill')
    df['dpanom'] = df['dpanom'].interpolate(method='linear').fillna(method='bfill')
    df['windspd'] = df['windspd'].interpolate(method='linear').fillna(method='bfill')
    df['uanom'] = df['uanom'].interpolate(method='linear').fillna(method='bfill')
    df['vanom'] = df['vanom'].interpolate(method='linear').fillna(method='bfill')
    vars = ['windgust', 'tempanom', 'stnpanom',
            'dpanom', 'windspd', 'uanom', 'vanom']
    
    #for idx, tmpdf in df.groupby('date'):
    #    scaler = StandardScaler()
    #    scalevals = scaler.fit_transform(tmpdf[vars].values)
    #    df.loc[df['date'] == idx, vars] = scalevals

    df['stnNum'] = stnNum
    df.reset_index(inplace=True)
    df.set_index(['stnNum', 'date'], inplace=True)

    return df

Load all the events into a single dataframe. We'll then pick out the events based on whether they are in the training set or the test set, using the index from the storm classification data

In [18]:
dflist = []
for stn in stndf.index:
    df = loadData(stn)
    dflist.append(df)

alldf = pd.concat(dflist)
alldf['idx'] = alldf.index

Split the event data into training and test datasets. This is simple because we set the index of both the event data and the storm classification data to be the station number and date of the event. 

The event data needs to be reshaped into a 3-d array (for input into the classifier), where there are n events in the first dimension. The second dimension represents the time (-60 minutes to +60 minutes) and the third dimension represents the different variables. We can potentially vary the list of variables used, but we start with the wind gust and the anomalies of temperature, dewpoint and station pressure.

In [19]:
eventdf_train = alldf.loc[train_storms.index]
eventdf_test = alldf.loc[test_storms.index]

vars = ['windgust', 'tempanom', 'stnpanom', 'dpanom']
scaler = StandardScaler()
eventdf_train[vars] = scaler.fit_transform(eventdf_train[vars].values)
eventdf_test[vars] = scaler.transform(eventdf_test[vars].values)

nvars = len(vars)
X = eventdf_train.reset_index().set_index(['idx', 'tdiff'])[vars]
XX = np.moveaxis(X.values.reshape((ntrain, 121, nvars)), 1, -1)

X_test = eventdf_test.reset_index().set_index(['idx', 'tdiff'])[vars]
XX_test = np.moveaxis(X_test.values.reshape((200, 121, nvars)), 1, -1)

fulltest = alldf.loc[stormdf.index].reset_index().set_index(['idx', 'tdiff'])[vars]
fulltestarray = np.moveaxis(fulltest.values.reshape((len(stormdf), 121, nvars)), 1, -1)
fully = np.array(list(stormdf.loc[fulltest.reset_index()['idx'].unique()]['stormType'].values))

y = np.array(train_storms['stormType'].values)

Here's where we train and run the classifier on the test event set. The classifier is fitted to the training data (`XX`), then we predict the class of the test data (`XX_test`). Results are compared to the visual classification of the test data.

In [20]:
rocket = RocketClassifier(num_kernels=10000)
rocket.fit(XX, y)
y_pred = rocket.predict(XX_test)
results = pd.DataFrame(data={'prediction':y_pred, 'visual':test_storms['stormType']})
score = rocket.score(XX_test, test_storms['stormType'])
print(f"Accuracy of the classifier: {score}")
colorder = ['Synoptic storm', 'Synoptic front', 'Storm-burst',
            'Thunderstorm', 'Front up', 'Front down',
            'Spike', ]
pd.crosstab(results['visual'], results['prediction']).reindex(colorder)[colorder]

Accuracy of the classifier: 0.78


prediction,Synoptic storm,Synoptic front,Storm-burst,Thunderstorm,Front up,Front down,Spike
visual,,,,,,,
Synoptic storm,55,0,3,0,0,0,0
Synoptic front,2,29,0,0,2,0,0
Storm-burst,10,0,9,1,0,0,0
Thunderstorm,1,4,4,23,8,0,0
Front up,0,4,0,0,14,0,0
Front down,0,1,0,0,0,8,0
Spike,2,0,0,0,0,0,18


Now we will run the classification on all storms. Firstly, we run with 2000 random convolution kernels. This is an efficient and preliminary analysis of the performance of the classification algorithm.

In [15]:
rocket = RocketClassifier(num_kernels=2000)
rocket.fit(fulltestarray, fully)
newclass = rocket.predict(fulltestarray)
results = pd.DataFrame(data={'prediction':newclass, 'visual':fully})
score = rocket.score(fulltestarray, fully)
print(f"Accuracy of the classifier: {score}")
colorder = ['Synoptic storm', 'Synoptic front', 'Storm-burst',
            'Thunderstorm', 'Front up', 'Front down',
            'Spike', 'Unclassified']
pd.crosstab(results['visual'], results['prediction']).reindex(colorder)[colorder]

Accuracy of the classifier: 0.9156626506024096


prediction,Synoptic storm,Synoptic front,Storm-burst,Thunderstorm,Front up,Front down,Spike,Unclassified
visual,,,,,,,,
Synoptic storm,295,3,0,0,2,0,3,0
Synoptic front,6,122,1,1,0,0,2,0
Storm-burst,10,1,138,2,1,0,3,0
Thunderstorm,7,3,1,134,2,0,1,0
Front up,9,4,0,4,84,1,2,0
Front down,3,0,0,1,1,45,0,0
Spike,6,2,0,1,1,0,82,0
Unclassified,0,0,0,0,0,0,0,12


The ROCKET classifier shows a tendency to predict all storm types are Synoptic storms - there are 18 visually classified convective storms that are predicted as Synoptic storm

Increasing the number of kernels substantially improves performance of the classification scheme. The score increases from 0.897 to 0.982. The number of incorrectly classified storms in the training dataset is reduced to just 18 out of 996 events. Of those, 12 are discussed in further detail.

In [16]:
rocket = RocketClassifier(num_kernels=10000)
rocket.fit(fulltestarray, fully)
newclass = rocket.predict(fulltestarray)
results = pd.DataFrame(data={'prediction':newclass, 'visual':fully})
score = rocket.score(fulltestarray, fully)
print(f"Accuracy of the classifier: {score}")
colorder = ['Synoptic storm', 'Synoptic front', 'Storm-burst',
            'Thunderstorm', 'Front up', 'Front down',
            'Spike', 'Unclassified']
pd.crosstab(results['visual'], results['prediction']).reindex(colorder)[colorder]
pd.crosstab(results['visual'], results['prediction']).reindex(colorder)[colorder].to_excel(pjoin(BASEDIR, 'events', 'crosstab.xlsx'))

Accuracy of the classifier: 0.9819277108433735


In [ ]:
results.loc[(results.prediction=='Spike') & (results.visual.isin(['Synoptic storm', 'Synoptic front', 'Storm-burst']))].index

In [ ]:
stormdf.loc[fulltest.reset_index()['idx'].unique()].iloc[[966, 967, 968, 969, 970, 975, 977]]

In [ ]:
results.loc[(results.prediction=='Synoptic storm') & (results.visual.isin(['Spike']))].index

In [ ]:
stormdf.loc[fulltest.reset_index()['idx'].unique()].iloc[[442, 443, 444, 445, 447]]

In [ ]:
proba = rocket.predict_proba(fulltestarray)
idx = [5, 4, 3, 6, 1, 0, 2, 7]
#proba[:, idx]
fig, ax = plt.subplots(figsize=(10,12))
plt.imshow(proba[:, idx])
ax.set_aspect(0.01)
plt.colorbar()

Load the station data for all stations

In [ ]:
from stndata import ONEMINUTESTNDTYPE, ONEMINUTESTNNAMES
allstnfile = r"X:\georisk\HaRIA_B_Wind\data\raw\from_bom\2022\1-minute\HD01D_StationDetails.txt"

allstndf = pd.read_csv(allstnfile, sep=',', index_col='stnNum',
                            names=ONEMINUTESTNNAMES,
                            keep_default_na=False,
                            converters={
                                'stnName': str.strip,
                                'stnState': str.strip
                            })

In [ ]:
alldatadflist = []
for stn in allstndf.index:
    try:
        df = loadData(stn)
    except FileNotFoundError:
        pass  #print(f"No data for station: {stn}")
    else:
        alldatadflist.append(df)

alldatadf = pd.concat(alldatadflist)
alldatadf['idx'] = alldatadf.index

In [ ]:
allX = alldatadf.reset_index().set_index(['idx', 'tdiff'])[vars]

Remove any events with less than 2 hours of observations, and any remaining events with missing data

In [ ]:

naidx = []
for ind, tmpdf in allX.groupby(level='idx'):
    #print(len(tmpdf))
    if len(tmpdf) < 121:
        naidx.append(ind)
        print(ind, len(tmpdf))
    if tmpdf.isna().sum().sum() > 0:
        # Found NA values in the data (usually dew point)
        naidx.append(ind)

allXupdate = allX.drop(naidx, level='idx')

Reshape the data for input to the ROCKET classifier:

In [ ]:
nstorms = int(len(allXupdate)/121)
vars = ['windgust', 'tempanom', 'stnpanom', 'dpanom']
nvars = len(vars)
#allX = alldatadf.reset_index().set_index(['idx', 'tdiff'])[vars]
allXX = np.moveaxis(allXupdate.values.reshape((nstorms, 121, nvars)), 1, -1)

Run the classifier

In [ ]:
stormclass = rocket.predict(allXX)

In [ ]:
len(stormclass)

In [ ]:
outputstormdf = pd.DataFrame(data={'stormType': stormclass}, index=allXupdate.index.get_level_values(0).unique())

In [ ]:
outputstormdf.stormType.value_counts().plot(kind='bar')

In [ ]:
outputstormdf.stormType.value_counts()

In [ ]:
allXupdate['idx'] = allXupdate.index.get_level_values(0)

In [ ]:


synidx = outputstormdf[outputstormdf['stormType']=="Synoptic storm"].index
syfidx = outputstormdf[outputstormdf['stormType']=="Synoptic front"].index
sbidx = outputstormdf[outputstormdf['stormType']=="Storm-burst"].index

tsidx = outputstormdf[outputstormdf['stormType']=="Thunderstorm"].index
fuidx = outputstormdf[outputstormdf['stormType']=="Front up"].index
fdidx = outputstormdf[outputstormdf['stormType']=="Front down"].index

ucidx = outputstormdf[outputstormdf['stormType']=="Unclassified"].index
spidx = outputstormdf[outputstormdf['stormType']=="Spike"].index

In [ ]:
synevents = allXupdate[allXupdate.index.get_level_values('idx').isin(synidx)]
syfevents = allXupdate[allXupdate.index.get_level_values('idx').isin(syfidx)]
sbevents = allXupdate[allXupdate.index.get_level_values('idx').isin(sbidx)]


tsevents = allXupdate[allXupdate.index.get_level_values('idx').isin(tsidx.values)]
fuevents = allXupdate[allXupdate.index.get_level_values('idx').isin(fuidx.values)]
fdevents = allXupdate[allXupdate.index.get_level_values('idx').isin(fdidx.values)]

ucevents = allXupdate[allXupdate.index.get_level_values('idx').isin(ucidx.values)]
spevents = allXupdate[allXupdate.index.get_level_values('idx').isin(spidx.values)]

In [ ]:
meansyn = synevents.groupby('tdiff').mean().reset_index()
meansyf = syfevents.groupby('tdiff').mean().reset_index()
meansb = sbevents.groupby('tdiff').mean().reset_index()

meants = tsevents.groupby('tdiff').mean().reset_index()
meanfu = fuevents.groupby('tdiff').mean().reset_index()
meanfd = fdevents.groupby('tdiff').mean().reset_index()

meanuc = ucevents.groupby('tdiff').mean().reset_index()
meansp = spevents.groupby('tdiff').mean().reset_index()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patheffects
pe = patheffects.withStroke(foreground="white", linewidth=5)

def plotEvent(df):
    fig, ax = plt.subplots(figsize=(12,8))
    #ax = fig.add_axes([0, 0, 1, 1])
    #ax2 = ax.twinx()
    axt = ax.twinx()
    axp = ax.twinx()
    ax.set_zorder(1)
    ax.patch.set_visible(False)
    lnt = axt.plot(df.tdiff, df.tempanom, label=r"Temperature anomaly [$^o$C]",
             color='r', marker='^', markerfacecolor="None", lw=2, path_effects=[pe], zorder=1,
             markevery=5)
    lnd = axt.plot(df.tdiff, df.dpanom, color='orangered', marker='.', markerfacecolor="None",
             lw=1, path_effects=[pe], zorder=1, markevery=5, label=r"Dew point anomaly [$^o$C]")
    
    lnw = ax.plot(df.tdiff, df.windgust, label="Gust wind speed [km/h]", 
            lw=3, path_effects=[pe], markerfacecolor="None",zorder=100)
    lnp = axp.plot(df.tdiff, df.stnpanom, color='purple', lw=2, path_effects=[pe],
             ls='--', label='Station pressure anomaly [hPa]')

    #axt.spines['right'].set_position(("axes", 1.075))
    axt.spines[['right']].set_color('r')
    axt.yaxis.label.set_color('r')
    axt.tick_params(axis='y', colors='r')
    axt.set_ylabel(r"Temperature/dewpoint anomaly [$^o$C]")

    ax.set_ylabel("Gust wind speed [km/h]")

    axp.spines[['right']].set_position(('axes', 1.075))
    axp.spines[['right']].set_color('purple')
    axp.yaxis.label.set_color('purple')
    axp.tick_params(axis='y', colors='purple')
    axp.set_ylabel("Pressure anomaly [hPa]")

    gmin, gmax = ax.get_ylim()
    pmin, pmax = axp.get_ylim()
    tmin, tmax = axt.get_ylim()
    ax.set_ylim((0, max(gmax, 100)))
    ax.set_xlabel("Time from gust peak(minutes)")
    axp.set_ylim((min(-2.0, pmin), max(pmax, 2.0)))
    axt.set_ylim((min(-2.0, tmin), max(tmax, 2.0)))
    #ax2.set_ylim((0, 360))
    #ax2.set_yticks(np.arange(0, 361, 90))
    #axr.set_ylim((0, 100))
    #ax.set_title(meants.index[0])
    ax.grid(True)
    #ax2.grid(False)
    axt.grid(False)
    axp.grid(False)
    
    lns = lnw + lnt + lnd + lnp
    labs = [l.get_label() for l in lns]
    
    ax.legend(lns, labs, )
    #axr.grid(False)

In [ ]:
plotEvent(meants)

In [ ]:
plotEvent(meanfu)

In [ ]:
plotEvent(meanfd)

In [ ]:
plotEvent(meansyn)

In [ ]:
plotEvent(meansyf)

In [ ]:
plotEvent(meansb)

In [ ]:
plotEvent(meansp)

In [ ]:
plotEvent(meanuc)